# M115 - Image Analysis and Processing, Assignment 2 (Notebook 1)

---

In this assignment, an intelligent system was developed to detect pneumonia in chest X-ray images, utilising a dataset available at [Kaggle Chest X-Ray Images (Pneumonia) dataset](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia). The task involved the creation of two sorts of algorithms; classical Machine Learning and Deep Learning ones. This notebook covers the Classical ML aspect, and all the preprocessing thereof.


This coursework is submitted as part of the requirements for the Image Analysis and Processing (M115) course during the spring semester of 2023, in the DSIT's Master degree programme at the National and Kapodistrian University of Athens. The author of this project is

- Michael Darmanis (SID: 7115152200004).

Parts of code presented in this [notebook](https://www.kaggle.com/code/hosen42/pneumonia-detection-using-traditional-ml) have been used.

The notebook was executed in Kaggle, so bear that in mind in case any issues arise while rerunning parts of the code locally (or in Google Colab, for that matter).

Whatever the case may be, it is essential to **run all the necessary libraries (following four code cells) beforehand**, otherwise the instance of matplotlib will not function properly. It is also advised that all cells are run in of appearnce because many exercise are dependent (variable or otherwise) from previously calculated results or created instances.

In [ ]:
# Dependencies for rendering text of matplotlib in LaTeX
!sudo apt update -y
!sudo apt install -y cm-super dvipng texlive-latex-extra texlive-latex-recommended

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2
from PIL import Image


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, matthews_corrcoef, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import confusion_matrix


from sklearn.utils import shuffle as shf
import pickle
import os
import glob as gb


import cv2
import skimage
from skimage import feature, filters
from tqdm import tqdm

# Consider aesthetics and consistency
plt.rcParams.update({
        "text.usetex": True,
        "font.size": 15,
        'mathtext.default': 'regular',
        'axes.titlesize': 16,
        "axes.labelsize": 16,
        "legend.fontsize": 15,
        "xtick.labelsize": 15,
        "ytick.labelsize": 15,
        'figure.titlesize': 16,
        'figure.figsize': (12, 7),
        'text.latex.preamble': r'\usepackage{amsmath,amssymb}',
        "font.family": "serif",
        "font.serif": "computer modern roman",
        })

In [ ]:
def show_random_images():
    path_random_normal = random.choice(train_normal)
    path_random_pneumonia = random.choice(train_pneumonia)
    
    fig = plt.figure(figsize=(10, 10))
    
    ax1 = plt.subplot(1, 2, 1)
    ax1.imshow(Image.open(path_random_normal).convert("LA"))
    ax1.set_title("Normal X-ray")
    
    ax2 = plt.subplot(1, 2, 2)
    ax2.imshow(Image.open(path_random_pneumonia).convert("LA"))
    ax2.set_title("Pneumonia X-ray")

    
def print_metrics(y_pred, y_train, yt_pred, y_test):
    print('Train data metrics:')
    print('Balanced accuracy score: ', balanced_accuracy_score(y_train, y_pred))
    print('F1 score: ', f1_score(y_train, y_pred))
    print('Precison: ', precision_score(y_train, y_pred))
    print('Recall: ', recall_score(y_train, y_pred))
    print('MCC',  matthews_corrcoef(y_train, y_pred))
    print()
    print('Test data metrics:')
    print('Balanced accuracy score: ', balanced_accuracy_score(y_test, yt_pred))
    print('F1 score: ', f1_score(y_test, yt_pred))
    print('Precison: ', precision_score(y_test, yt_pred))
    print('Recall: ', recall_score(y_test, yt_pred))
    print('MCC',  matthews_corrcoef(y_test, yt_pred))


def plot_confusion_matrix(predictions, y_test, title):
    labels = ['Normal', 'Pnuemonia']
    
    cm = confusion_matrix(y_test,predictions)
    cm = pd.DataFrame(cm , index = labels , columns = labels)
    
    plt.figure()
    sns.heatmap(cm, cmap="YlGnBu", linecolor = 'black' , linewidth = 1,
                annot = True, fmt='', xticklabels = labels,
                yticklabels = labels)
    
    plt.title(title, fontsize = 20)
    plt.xlabel('Predicted', fontsize = 15)
    plt.ylabel('Actual', fontsize = 15)
    
    plt.savefig(f'/kaggle/working/{title.replace(" ", "_")}_cm.pdf',
                format='pdf', bbox_inches='tight', dpi=300, transparent=True)
    plt.show()
    
    
def evaluate_classifiers(X_test, y_test):
    # Create a dictionary to store your classifiers
    classifiers = {'Logistic Regression': lg, 'Decision Tree': dtc,
                   'Random Forest': rfc, 'SVM': svm}

    # Initialize an empty dictionary to store the scores for each classifier
    scores = {'Classifier': [], 'F1': [], 'Balanced Accuracy': [], 'Precision': [],
              'MCC': [], 'Recall': []}

    # Iterate over the classifiers
    for clf_name, clf in classifiers.items():
        y_pred = clf.predict(X_test)

        # Calculate scores
        f1 = f1_score(y_test, y_pred, average='weighted') 
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        mcc = matthews_corrcoef(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='weighted')

        # Add scores to the scores dictionary
        scores['Classifier'].append(clf_name)
        scores['F1'].append(f1)
        scores['Balanced Accuracy'].append(balanced_accuracy)
        scores['Precision'].append(precision)
        scores['MCC'].append(mcc)
        scores['Recall'].append(recall)

    return pd.DataFrame(scores)

## Data exploration

Visualise and understand the dataset's properties, including data distribution, class imbalance, and data quality.

In [ ]:
train_normal = gb.glob("../input/chest-xray-pneumonia/chest_xray/train/NORMAL/*")
train_pneumonia = gb.glob("../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/*")

test_normal = gb.glob("../input/chest-xray-pneumonia/chest_xray/test/NORMAL/*")
test_pneumonia = gb.glob("../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/*")

val_normal = gb.glob("../input/chest-xray-pneumonia/chest_xray/val/NORMAL/*")
val_pneumonia = gb.glob("../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/*")

In [ ]:
print('Train normal:', len(train_normal), '   ', 'Train pneumonia:', len(train_pneumonia))
print('Test normal:', len(test_normal), '     ', 'Test pneumonia:', len(test_pneumonia))
print('Validation normal:', len(val_normal), ' ', 'Validation pneumonia:', len(val_pneumonia))

First of all, the validation size is insignificant. It will therefore be concatenated with the training data and, should validation data be required, the training data will be randomly split at a 9-1 analogy.

There also exists an imbalance of classes. To mitigate this issue, the `class_weight` parameter will be employed with the value `balanced`, for all ML models used. The `class_weight` argument modifies the loss function during training by assigning a higher penalty for misclassifying the minority class.

In the face of class imbalance, the implementation of metrics that evaluate performance across all classes, also becomes crucial. Therefore the following metrics will be employed throughout:
- Balanced accuracy
- F1 score
- Precision
- Recall
- Matthews correlation coefficient

Balanced Accuracy, which computes the average recall for each class, provides an effective solution for imbalanced datasets due to its indifference towards the majority class. The F1 score, representing the harmonic mean of precision and recall, is also valuable, especially when the positive class bears more significance and a balance between Precision (representing the accuracy of positive predictions) and Recall (indicating the detection rate of all positive instances) is sought.

Additionally, Precision and Recall are key metrics that permit model fine-tuning in specific directions. Precision, the ratio of true positive predictions to the total predicted positives, becomes vital when the reduction of false positives is the goal. Conversely, Recall, the ratio of true positive predictions to all actual positive instances, is pivotal when the maximization of positive instance detection is the objective. The Matthews Correlation Coefficient (MCC), an all-encompassing measure for binary classifications, takes into account both true and false positives and negatives, thereby offering a balanced metric, particularly useful when the classes are of vastly different sizes.

Within the framework of the currect project, these metrics will provide an satisfactory evaluation of the models' performance; permitting necessary adjustments for managing class imbalance.

Next, random images from both train and test sets are checked.

In [ ]:
show_random_images()

In [ ]:
show_random_images()
del train_normal, test_normal, val_normal
del train_pneumonia, test_pneumonia, val_pneumonia

Upon inspection, the images seems to vary in both size and rotation (slightly); this will be born to mind when preprocessing the data and when fitting the ML models.

## On-the-spot attempt

For a first, on-the-spot, attempt; a basic preprocessing will be perfomed. More particularly, the images will be reduced to 300 by 300 pixels and a normalisation & Principal Component Analysis will be applied.

<div style="text-align:center">
<img src="https://i.imgur.com/gW78vT9.png" alt="workflow" width="500" height="600"/>
</div>

Four classifier models will be trained: Linear Regression, Random Forest, Decision Tree, and a Support Vector Machine. The model yielding the best output in terms of the chosen metrics will then be parameter-tuned using a grid search.

In [ ]:
# Assign class of the X-rays based on folder name
code = {'NORMAL':0 ,'PNEUMONIA':1}
def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x

In [ ]:
# Directories that contain the train and validation images set
paths = ['../input/chest-xray-pneumonia/chest_xray/train/', 
         '../input/chest-xray-pneumonia/chest_xray/val/']

X_train = []
y_train = []

for trainpath in paths:
    for folder in  os.listdir(trainpath) : 
        files = gb.glob(pathname= str( trainpath + folder + '/*.jpeg'))
        for file in files: 
            image = cv2.imread(file)
            # Resize images to 300 x 300 pixels
            image_array = cv2.resize(image , (300, 300))
            X_train.append(list(image_array))
            y_train.append(code[folder])

X_train = np.asarray(X_train)
X_train = X_train.astype(np.float32)
np.save('X_train', X_train)
del X_train

y_train = np.asarray(y_train)
y_train = y_train.astype(np.float32)
np.save('y_train', y_train)
del y_train

In [ ]:
# Directory that contains the test images set
testpath='../input/chest-xray-pneumonia/chest_xray/test/'

X_test = []
y_test = []
for folder in  os.listdir(testpath) : 
    files = gb.glob(pathname= str( testpath + folder + '/*.jpeg'))
    for file in files: 
        image = cv2.imread(file)
        # Resize images to 300 x 300 pixels
        image_array = cv2.resize(image , (300, 300))
        X_test.append(list(image_array))
        y_test.append(code[folder])

X_test = np.asarray(X_test)
X_test = X_test.astype(np.float32)
np.save('X_test',X_test)
del X_test

y_test = np.asarray(y_test)
y_test = y_test.astype(np.float32)
np.save('y_test',y_test)
del y_test

In [ ]:
# Load datasets
X_train = np.load('./X_train.npy')
X_test = np.load('./X_test.npy')
y_train = np.load('./y_train.npy')
y_test = np.load('./y_test.npy')

In [ ]:
# Flatten the images into a 2d array, for model training and testing
X_train = X_train.reshape([-1, np.product((300, 300, 3))])
X_test = X_test.reshape([-1, np.product((300, 300, 3))])

In [ ]:
# Shuffle train and test datasets in a consistent way
X_train, y_train = shf(X_train, y_train, random_state=15)
X_test, y_test = shf(X_test, y_test, random_state=15)

In [ ]:
# Normalise train and test datasets
sc = StandardScaler(copy=False)
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
# Principal component analysis with 95% retained variance 
pca = PCA(.95)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

print('Number of components after PCA: ' + str(pca.n_components_))

In [ ]:
# Make an instance for each and every classification algorithm
# notice the class_weight option set to balanced
lg  = LogisticRegression(class_weight='balanced')
dtc = DecisionTreeClassifier(class_weight='balanced')
rfc = RandomForestClassifier(class_weight='balanced')
svm = SVC(class_weight='balanced')

In [ ]:
# Fitting each vanilla model
lg.fit(X_train, y_train)
dtc.fit(X_train, y_train)
rfc.fit(X_train, y_train)
svm.fit(X_train, y_train)

In [ ]:
# Show evaluation metrics for all algorithms
df_scores = evaluate_classifiers(X_test, y_test)
print(df_scores)

Based on all these metrics, it would appear that the SVM model offers the best performance on the unbalanced X-ray dataset. Therefore, a brute-force parameter tuning will be performed based on the F1 score.

In [ ]:
# Defining parameter range
param_grid_svm = {'C': [0.1, 1, 10],  
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf', 'linear', 'poly']} 

grid_svm = GridSearchCV(svm, param_grid_svm,
                        refit = True, verbose = 3,
                        scoring='f1')
  
# Fitting the model for grid search on the training data
grid_svm.fit(X_train, y_train)

# Inspect the best parameters found by GridSearchCV
print('Best parameters for SVM:', grid_svm.best_params_)

In [ ]:
# Get the best model
best_svm = grid_svm.best_estimator_

# Calculate and print the metrics
print_metrics(best_svm.predict(X_train), y_train,
              best_svm.predict(X_test), y_test)

In [ ]:
plot_confusion_matrix(best_svm.predict(X_test), y_test, 'Optimised SVM Model')

In [ ]:
del X_train, y_train, X_test, y_test, pca, sc

## Feature-extraction attempt

As an alternative, and more nuanced approach, traditional image processing methods will be applied. The aim now will be that of extracting features for the classification task.

Pneumonia can readily be observed by lung oppacities, therefore, segmenting and focusing only on the lung part can be of importance (see this [notebook](https://www.kaggle.com/code/mikedarm/what-are-lung-opacities)).

This will be accomplished by using numerous image processing techniques. Here's a brief overview of the steps involved and their importance:

1. **Equalisation**: Image equalisation enhances the contrast of the lungs and accentuates the presence of opacity. This will most likely improve contrast which facilitates subsequent feature extraction.

2. **Image Sharpening**: High pass filtering is used for image sharpening because it reveals more detail compared to the unmask method. Sharpening the image allows for the extraction of more precise and detailed features.

3. **Otsu Thresholding**: Otsu thresholding technique provides smoother edges and better lung segment isolation, which is why it's used in this pipeline.

4. **Edge Detection**: For edge detection, the Sobel filter is chosen as it extracts edges more effectively than the Canny filter in similar [projects](https://doi.org/10.1038/s41551-021-00787-w).

5. **Moment Calculation**: Once the lung segment is identified, the center of the moment is calculated as a feature for prediction. As many X-ray images do not have the same dimensions, this feature can pose a problem however.

6. **Rotation and Scale Invariance**: Some images may present subjects in slightly rotated positions or different sizes. Therefore, the center of the moment is needed to be invariant to rotation and scale. Hu moments are chosen for this purpose ([Digital Image Processing](https://dl.acm.org/doi/book/10.5555/22881)), and to make comparison easier, the moments are logged. The third moment, which depends on the other moments, and the seventh moment, which distinguishes mirror images, are dropped, as no flipped images were observed in the dataset.

The adopted pipeline is seen in the following picture:

<div style="text-align:center">
<img src="https://i.imgur.com/5d3W9p5.png" alt="workflow" width="500" height="600"/>
</div>

The selected features for building a classifier for pneumonia detection, therefore, are:
* Mean and Standard Deviation of unenhanced image
* Area of opacity
* Perimeter of visible lung regions
* Irregularity index
* Equivalent diameter
* Hu moments (5 out of 7)

The same classifiers will be used as previously, and the best vanilla model will be parameter-tuned.

In [ ]:
def area(img):
    # Binarized image as input
    return np.count_nonzero(img)

def perimeter(img):
    # Edges of the image as input
    return np.count_nonzero(img)

def irregularity(area, perimeter):
    # Area and perimeter of the image as input (i.e. compactness)
    I = (4 * np.pi * area) / (perimeter ** 2)
    return I

def equiv_diam(area):
    # Area of image as input
    ed = np.sqrt((4 * area) / np.pi)
    return ed

def get_hu_moments(contour):
    # Hu moments except 3rd and 7th (5 values)
    M = cv2.moments(contour)
    hu = cv2.HuMoments(M).ravel().tolist()
    del hu[2], hu[-1]
    log_hu = [-np.sign(a)*np.log10(np.abs(a)) for a in hu]
    return log_hu

In [ ]:
def extract_features(img):
    """
    The function  carries out the steps mentioned above on an input image.
    It begins with basic statistical calculations (mean and standard deviation),
    applies image processing techniques such as histogram equalisation,
    sharpening, thresholding, and edge detection.
    Then, it finds contours and selects the one with the most points.
    The function then calculates various features such as area, perimeter,
    irregularity, equivalent diameter, and Hu moments from the selected contour.
    It then returns these calculated features for the given input image.
    """
    mean = img.mean()
    std_dev = img.std()
    
    # Histogram equalisation
    equalized = cv2.equalizeHist(img)
    
    # Sharpening
    hpf_kernel = np.full((3, 3), -1)
    hpf_kernel[1,1] = 9
    sharpened = cv2.filter2D(equalized, -1, hpf_kernel)
    
    # Thresholding
    ret, binarized = cv2.threshold(cv2.GaussianBlur(sharpened, 
                                        (7, 7), 0), 0, 255, 
                                   cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Edge detection
    edges = skimage.filters.sobel(binarized)
    
    # Moments from contours
    contours, hier = cv2.findContours((edges * 255).astype('uint8'), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    select_contour = sorted(contours, key=lambda x: x.shape[0], reverse=True)[0]
    
    # Feature extraction
    ar = area(binarized)
    per = perimeter(edges)
    irreg = irregularity(ar, per)
    eq_diam = equiv_diam(ar)
    hu = get_hu_moments(select_contour)
    
    return [mean, std_dev, ar, per, irreg, eq_diam, *hu]

In [ ]:
# Directories that contain the train and validation images set
# (class labels are available from previous part)
paths = ['../input/chest-xray-pneumonia/chest_xray/train/', 
         '../input/chest-xray-pneumonia/chest_xray/val/']

X_train = []

for trainpath in paths:
    for folder in  os.listdir(trainpath) : 
        files = gb.glob(pathname= str( trainpath + folder + '/*.jpeg'))
        for file in files: 
            image = cv2.imread(file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Feature extraction
            features = extract_features(image)
            X_train.append(features)

X_train = np.asarray(X_train)
X_train = X_train.astype(np.float32)
np.save('X_train_fe', X_train)
del X_train

In [ ]:
# Directory that contain the test images set
# (class labels are available from previous part)
testpath='../input/chest-xray-pneumonia/chest_xray/test/'

X_test = []
for folder in  os.listdir(testpath) : 
    files = gb.glob(pathname= str( testpath + folder + '/*.jpeg'))
    for file in files: 
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
        # Feature extraction
        features = extract_features(image)
        X_test.append(features)

X_test = np.asarray(X_test)
X_test = X_test.astype(np.float32)
np.save('X_test_fe',X_test)
del X_test

In [ ]:
# Load datasets
X_train = np.load('./X_train_fe.npy')
X_test = np.load('./X_test_fe.npy')
y_train = np.load('./y_train.npy')
y_test = np.load('./y_test.npy')

In [ ]:
# Shuffle train and test datasets in a consistent way
X_train, y_train = shf(X_train, y_train, random_state=15)
X_test, y_test = shf(X_test, y_test, random_state=15)

In [ ]:
# Normalise datasets
sc = StandardScaler(copy=False)
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
# Fitting each vanilla model
lg.fit(X_train, y_train)
dtc.fit(X_train, y_train)
rfc.fit(X_train, y_train)
svm.fit(X_train, y_train)

In [ ]:
# Show evaluation metrics for all algorithms
df_scores = evaluate_classifiers(X_test, y_test)
print(df_scores)

Considering the above metrics as a whole, Logistic Regression is the best model. However, the difference between Logistic Regression and SVM is quite small. Also the SVM's slightly higher precision might make it a good alternative, especially if avoiding false positives is a priority.

For the time being, the Logistic Regresion will be parameter-tuned.

In [ ]:
# Defining parameter range
param_grid_lg = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': list(range(100, 800, 100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

grid_lg = GridSearchCV(lg, param_grid=param_grid_lg, refit = True,
                       verbose = 3, cv=5, scoring='f1')
  
# Fitting the model for grid search on the training data
grid_lg.fit(X_train, y_train)

# Inspect the best parameters found by GridSearchCV
print('Best parameters for SVM:', grid_lg.best_params_)

In [ ]:
# Get the best model
best_lg = grid_lg.best_estimator_

# Calculate and print the metrics
print_metrics(best_lg.predict(X_train), y_train, best_lg.predict(X_test), y_test)

In [ ]:
plot_confusion_matrix(best_lg.predict(X_test), y_test, 'Optimised Logistic Regression Model')

It is clear that using this pipeline, the best model (logistic regression) performs better than the one from the on-the-spot approach (SVM). The logistic regression may hit more false-possitives, however, it has greatly reduced the true-negatives (by 46).

The ideal model, therefore, for the chosen intelligent system will be the Logistic Regression using the proposed image preprocessing pipeline.

## Best ML Intelligent System

Download an X-ray image from [medschool.co](https://medschool.co/) and test the best classic Ml model.

In [ ]:
!wget https://medschool.co/images/detail/cxr/consolidation-rml.jpg

In [ ]:
image = cv2.imread('/kaggle/working/consolidation-rml.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
features = extract_features(image)

pred = best_lg.predict([features])

if pred == 1:
    print('Patient is infected with pneumonia')
else:
    print('Patient is normal')